In [1]:
import pandas as pd
import numpy as np;


import matplotlib.pyplot as plt
import seaborn as sns

### Read in filter_data csv

In [2]:
df = pd.read_csv('ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv')
df = df.sort_values('month')
df.shape

/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_16220/285384369.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv')


(717069, 50)

In [3]:
df['month'] = pd.to_datetime(df['month'])
# filter for the last 5 years
df1 = df[df['month'] > '2015-01-01']
df.shape

(717069, 50)

In [4]:
# use month and lease_commence_date to calculate the age of the flat
df1['month'] = pd.to_datetime(df1['month'])
df1['lease_commence_date'] = pd.to_datetime(df1['lease_commence_date'])

df1.loc[:,'age_of_flat'] = df1['month'].dt.year - df1['lease_commence_date'].dt.year

df1 = df1.drop(columns=['lease_commence_date'])

df1 = df1.sort_values('month')

/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_16220/1415232116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['month'] = pd.to_datetime(df1['month'])
/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_16220/1415232116.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['lease_commence_date'] = pd.to_datetime(df1['lease_commence_date'])
/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_16220/1415232116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a 

In [5]:
columns_of_interest = ['avg_storey_range', 'floor_area_sqm', 'normalized_resale_price',
       'total_dwelling_units', 'vacancy', 'commercial', 
       'mrt_interchange', 'flat_model', 'town', 'age_of_flat']



In [6]:
threshold = 0.05*len(df1)

In [7]:
# only keep relevant columns
df1 = df1[columns_of_interest]

## Let's look at Cat. Variables first

In [8]:
df1['flat_model'] = df1['flat_model'].str.lower()
df1['town'] = df1['town'].str.lower()

In [9]:
# find the flat_models that have less than 10,000
other_models = df1['flat_model'].value_counts()[df1['flat_model'].value_counts() < threshold].index
other_models = list(other_models)

# replace the flat_models that have less than 10,000 with 'other'
df1['flat_model'] = df1['flat_model'].replace(other_models, 'other')


In [10]:
# find the flat_types that have less than 10,000
other_models = df1['town'].value_counts()[df1['town'].value_counts() < threshold].index
other_models = list(other_models)

# replace the flat_models that have less than 10,000 with 'other'
df1['town'] = df1['town'].replace(other_models, 'other')


### Numeric Variables

In [11]:
columns_to_be_scaled = ['avg_storey_range', 
       'floor_area_sqm',
       'normalized_resale_price', 
       'total_dwelling_units', 'vacancy']


In [12]:
# min_max scaling for all numerical columns
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#df1[columns_to_be_scaled] = scaler.fit_transform(df1[columns_to_be_scaled])

scaling_dic = {}

for col in columns_to_be_scaled:
  col_min = min(df1[col])
  col_max = max(df1[col])
  df1[col] = df1[col].apply(lambda x: (x-col_min)/(col_max-col_min))
  scaling_dic[col] = (col_min, col_max)


In [13]:
scaling_dic

## store somewhere

{'avg_storey_range': (2.0, 50.0),
 'floor_area_sqm': (31.0, 280.0),
 'normalized_resale_price': (154209.1203140869, 1540198.2919407086),
 'total_dwelling_units': (2, 570),
 'vacancy': (20, 110)}

In [14]:
df1.columns

Index(['avg_storey_range', 'floor_area_sqm', 'normalized_resale_price',
       'total_dwelling_units', 'vacancy', 'commercial', 'mrt_interchange',
       'flat_model', 'town', 'age_of_flat'],
      dtype='object')

# Summary

 - Grouped duplicate categories in Categorical variables e.g. Model A and model A
 - Grouped categories that appear less frequently to avoid overfitting
 - Min-Max scaling for numeric variables